In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os
import re
from sklearn.model_selection import cross_val_score  #交叉验证
from sklearn.svm import SVC  #支持向量机
from sklearn.metrics import accuracy_score  #混淆矩阵准确度
from sklearn.metrics import classification_report  #混淆矩阵分类性能评估报告
from sklearn.metrics import confusion_matrix    #混淆矩阵数组

## 说明：
本程序为使用dlib库128维特征距离数据集进行SVM情感分类的简单实验

In [3]:
trainDistance = np.load("data/trainDistance.npy")
testDistance = np.load("data/testDistance.npy")
DS_face_map = np.load("data/DS_face_map.npy")
DR_face_map = np.load("data/DR_face_map.npy")
DS_face = np.load("data/DS_face.npy",allow_pickle=True)
DR_face = np.load("data/DR_face.npy",allow_pickle=True)

print("DR_face_map: ", DR_face_map)
print("DS_face_map: ", DS_face_map)
print("DR_face: ", DR_face)
print("DS_face: ", DS_face)

DR_face_map:  [0 1 0 ... 1 0 1]
DS_face_map:  [0 0 0 ... 1 0 0]
DR_face:  ['smiling' 'serious' 'smiling' ... 'serious' 'smiling' 'serious']
DS_face:  ['smiling' 'smiling' 'smiling' ... 'serious' 'smiling' 'smiling']


不处理

In [11]:
#采用sklearn.svm.SVC，类似的有sklearn.svm.LinearSVC

clf_ovoLinear = SVC(kernel='rbf')
clf_ovoLinear.fit(trainDistance, DR_face)

SVC()

In [12]:
ovoLinear_face_pre = clf_ovoLinear.predict(testDistance)
ovoLinear_accuracy = accuracy_score(DS_face, ovoLinear_face_pre)
print('ovoLinear_accuracy = ', ovoLinear_accuracy)
print('分类结果报告：', '\n', classification_report(DS_face, ovoLinear_face_pre))

ovoLinear_accuracy =  0.6598196392785571
分类结果报告： 
               precision    recall  f1-score   support

       funny       0.00      0.00      0.00        63
     serious       0.71      0.68      0.69      1097
     smiling       0.60      0.69      0.64       836

    accuracy                           0.66      1996
   macro avg       0.44      0.45      0.45      1996
weighted avg       0.64      0.66      0.65      1996



C:\Users\Rama\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Rama\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Rama\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SMOTE过采样

In [4]:
from imblearn.over_sampling import SMOTE
imb_SMOTE = SMOTE(random_state = 0)
trainDistance_SMOTE, DR_face_SMOTE = imb_SMOTE.fit_resample(trainDistance, DR_face)
trainDistance_SMOTE.shape

(3129, 128)

In [5]:
SMOTE_face_dataframe = pd.DataFrame(data =  {'face':DR_face_SMOTE})
print(SMOTE_face_dataframe)
SMOTE_face_count = SMOTE_face_dataframe['face'].value_counts()
print(SMOTE_face_count)

         face
0     smiling
1     serious
2     smiling
3     smiling
4     serious
...       ...
3124  serious
3125  serious
3126  serious
3127  serious
3128  serious

[3129 rows x 1 columns]
smiling    1043
serious    1043
funny      1043
Name: face, dtype: int64


In [6]:
#采用sklearn.svm.SVC，类似的有sklearn.svm.LinearSVC

clf_ovoLinear_SMOTE = SVC(kernel='linear')
clf_ovoLinear_SMOTE.fit(trainDistance_SMOTE, DR_face_SMOTE)

SVC(kernel='linear')

In [8]:
ovoLinear_SMOTE_face_pre = clf_ovoLinear_SMOTE.predict(testDistance)
ovoLinear_SMOTE_accuracy = accuracy_score(DS_face, ovoLinear_SMOTE_face_pre)
print('ovoLinear_SMOTE_accuracy = ', ovoLinear_SMOTE_accuracy)
print('分类结果报告：', '\n', classification_report(DS_face, ovoLinear_SMOTE_face_pre))

ovoLinear_SMOTE_accuracy =  0.469438877755511
分类结果报告： 
               precision    recall  f1-score   support

       funny       0.05      0.54      0.09        63
     serious       0.70      0.48      0.57      1097
     smiling       0.65      0.45      0.53       836

    accuracy                           0.47      1996
   macro avg       0.47      0.49      0.40      1996
weighted avg       0.66      0.47      0.54      1996

